# KB차차차 사이트 크롤링 

## 1) 각자 carId 리스트 파일 불러오기

In [1]:
# import modules 

# - BeautifulSoup 관련 
from urllib.request import urlopen 
from bs4 import BeautifulSoup

# - time.sleep() 
import time 

# - dataframe 관련 
import pandas as pd

# - csv 관련 
import csv 

In [2]:
# 리스트 불러오기 (jiyeah.csv 예시 입니다. 본인 파일명으로 바꾸고 사용하기) 
# 리스트 읽기 (yejin_list 예시) 
jiyeah_list = list()
f = open("jiyeah.csv",'r')
rea = csv.reader(f)
for row in rea:
    jiyeah_list.append(row)
f.close

<function TextIOWrapper.close()>

In [3]:
# 리스트 안에 리스트가 들어가서 빼줄게요 
jiyeah_list = jiyeah_list[0]
# len(jiyeah_list) - 길이 확인 

In [4]:
jiyeah_list

['23007997',
 '23073400',
 '22888402',
 '22984607',
 '23040917',
 '23083524',
 '22919030',
 '23037265',
 '23004399',
 '23112228',
 '22858466',
 '22858545',
 '22858568',
 '22858677',
 '22858680',
 '22858847',
 '22858987',
 '22859138',
 '22859170',
 '22859287',
 '22859325',
 '22859394',
 '22859480',
 '22859487',
 '22859606',
 '22859678',
 '22859684',
 '22859823',
 '22859843',
 '22859925',
 '23104053',
 '22779456',
 '22732779',
 '22840538',
 '22346553',
 '22891001',
 '23001502',
 '22637104',
 '23086295',
 '22983481',
 '22859990',
 '22860004',
 '22860008',
 '22860317',
 '22860402',
 '22860496',
 '22860507',
 '22860511',
 '22860514',
 '22860527',
 '22860547',
 '22860696',
 '22860784',
 '22860792',
 '22860818',
 '22860819',
 '22860832',
 '22860852',
 '22860907',
 '22860920',
 '23008723',
 '22779456',
 '22584834',
 '22968819',
 '22678898',
 '22869436',
 '22874301',
 '22387561',
 '23048953',
 '23042337',
 '22860971',
 '22861042',
 '22861089',
 '22861094',
 '22861104',
 '22861219',
 '22861410',

## 2. 자동차 상세 정보 크롤링 
[columns]   
- Id(차량아이디), Country(국산/수입) : Id는 carSeq 리스트 사용, Country는 직접 입력   
- Name(차 이름), Price(가격-만원), Year(연식), Mile(주행거리-km), Fuel(연료), Loc(거래지역)
- Gear(변속기), MPG(연비-km / '정보' = '정보없음'), Kind(차종), Power(배기량-cc), Color(색상)
- **Opt1(옵션1), ...     
- **세금미납, 얍류, 저당

### 상세페이지에서 상세정보 리스트로 추출하는 함수 

In [5]:
domestic_carseq = jiyeah_list[0:5]

In [6]:
# 페이지에서 상세 정보 리스트로 추출하는 함수 
carid_list = []
name_list = []
price_list = []
year_list = []
mile_list = []
fuel_list = []
loc_list = []
gear_list = []
mpg_list = []
kind_list = []
power_list = []
color_list = []
optionlists=[]
sunroof_list = []
navi_list = []
smartkey_list = []
autolight_list = []
parksensor_list = []
leatherseat_list = []
heatseat_list = []
airseat_list = []
company_list = []


optionlists.append(sunroof_list)
optionlists.append(navi_list)
optionlists.append(smartkey_list)
optionlists.append(autolight_list)
optionlists.append(parksensor_list)
optionlists.append(leatherseat_list)
optionlists.append(heatseat_list)
optionlists.append(airseat_list)


def get_car_detail(url):
    
  
    url = "https://www.kbchachacha.com/public/car/detail.kbc?carSeq=" + str(carId)
    html = urlopen(url)
    bs_obj = BeautifulSoup(html, 'html.parser')
    
    carid = carId
    name = bs_obj.find("strong", {"class":"car-buy-name"}).text.split(")")[1]
    company = name.split(" ")[0]
    price = bs_obj.find("div", {"class":"car-buy-price"}).dd.text.strip()[:-2]
    
    txt_info = bs_obj.find("div", {"class":"txt-info"}).text.split("\n")
    year = txt_info[1]
    mile = txt_info[2][:-2]
    fuel = txt_info[3]
    loc = txt_info[4]
    
    detail_info = bs_obj.find("table", {"class":"detail-info-table"}).findAll("td")
    gear = detail_info[4].text
    mpg = detail_info[5].text.strip()[:-2]
    kind = detail_info[6].text
    power = detail_info[7].text[:-2]
    color = detail_info[8].text
    
    carid_list.append(carid)
    name_list.append(name)
    company_list.append(company)
    price_list.append(price)
    year_list.append(year)
    mile_list.append(mile)
    fuel_list.append(fuel)
    loc_list.append(loc)
    gear_list.append(gear)
    mpg_list.append(mpg)
    kind_list.append(kind)
    power_list.append(power)
    color_list.append(color)
    
    #옵션 추출 
    options = bs_obj.find("ul", {"class":"car-option-list"}).findAll("li")
    opt = ["썬루프","내비게이션","스마트키","오토라이트","주차감지센서","가죽시트","열선시트","통풍시트",]
        
    for j in range(len(opt)):
        for i in range(len(options)-1):
            if(opt[j]==options[i].text.split("(")[0]):
                optionlists[j].append("true")
                break
        if(opt[j]!=options[i].text):
            optionlists[j].append("false")



In [7]:
url = "https://www.kbchachacha.com/public/car/detail.kbc?carSeq=23007997" 

In [8]:
# 상세 페이지에서 상세 정보 데이터 가져오기 
# domestic_carseq --> 자기 carid가 들어있는 리스트 이름이 되면 됩니다!!!!!! 
country = [] # country - 국산 / 수입 리스트 
for i in range(len(domestic_carseq)):
    country.append("국산") # 자기 꺼에 따라 달라짐 
    carId = domestic_carseq[i]
    get_car_detail(url)
   

## 3. DataFrame 만들어서 csv로 내보내기  

1) 상세정보 리스트 만들기 (get_car_detail() 사용)    
2) 리스트 합쳐서 딕셔너리로 만들기    
3) 딕셔너리 -> 데이터프레임 변환    
4) 데이터프레임 -> csv로 내보내기     

In [9]:
# 딕셔너리 만들기 
cars_dict = {'Id' : carid_list,
                 'Country' : country, 
                 'Name' : name_list, 
                 'Company' : company_list,
                 'Price' : price_list,
                 'Year' : year_list,
                 'Mile' : mile_list,
                 'Fuel' : fuel_list, 
                 'Loc' : loc_list,
                 'Gear' : gear_list,
                 'MPG' : mpg_list,
                 'Kind' : kind_list,
                 'Power' : power_list,
                 'Color' : color_list,
                 'Sunroof' : sunroof_list,
                 'Navi' : navi_list,
                 'Smartkey' : smartkey_list,
                 'Autolight' : autolight_list,
                 'Parksensor' : parksensor_list,
                 'Leatherseat' : leatherseat_list,
                 'Heatseat' : heatseat_list,
                 'Airseat' : airseat_list}

In [10]:
cars_dict

{'Id': ['23007997', '23073400', '22888402', '22984607', '23040917'],
 'Country': ['국산', '국산', '국산', '국산', '국산'],
 'Name': ['기아 올 뉴 K73.0 LPI 렌터카 프레스티지 ',
  '르노코리아 XM31.3 Turbo Gasoline RE',
  '기아 스포티지 4세대디젤 2.0 2WD 프레스티지 ',
  '현대 그랜저IG 하이브리드프리미엄 ',
  '현대 베뉴가솔린 1.6 FLUX'],
 'Company': ['기아', '르노코리아', '기아', '현대', '현대'],
 'Price': ['2,110', '1,850', '1,350', '2,990만원\n2,980', '2,100'],
 'Year': ['19년03월(19년식)',
  '20년06월(20년식)',
  '16년10월(17년식)',
  '19년02월(19년식)',
  '20년06월(20년식)'],
 'Mile': ['67,325', '44,000', '145,899', '35,016', '17,230'],
 'Fuel': ['LPG', '가솔린', '디젤', '하이브리드(가솔린)', '가솔린'],
 'Loc': ['울산', '경기', '대전', '경기', '서울'],
 'Gear': ['오토', '오토', '오토', '오토', '오토'],
 'MPG': ['7.1', '13.7', '정보', '16.2', '13.3'],
 'Kind': ['대형', 'SUV', 'SUV', '대형', 'SUV'],
 'Power': ['2,999', '1,332', '1,995', '0', '1,598'],
 'Color': ['회색', '파랑색', '흰색', '검정색', '파랑(남색,곤색)'],
 'Sunroof': ['false', 'false', 'false', 'false', 'false'],
 'Navi': ['true', 'true', 'true', 'true', 'false'],
 'Smartkey': [

In [139]:
# 딕셔너리 -> 데이터프레임 만들기 (pandas 사용)

cars_df = pd.DataFrame(cars_dict)
#print(domestic_df)

In [140]:
# 데이터프레임 -> csv로 추출 (pandas 사용)

domestic_df.to_csv("domestic_5_new.csv")

In [ ]:
# ---- R에서 읽어서 분석 -------- # 